In [1]:
#2
#athena db config - aimodelpoll
#AAI-540 Group 3 FP

In [2]:
import boto3
import sagemaker
from pyathena import connect
import pandas as pd
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
ingest_create_athena_db_passed = False

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
database_name = "dsoaws"
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [4]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS dsoaws


/tmp/ipykernel_2654/2532319348.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [5]:
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_2654/1677879827.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws


In [7]:
s3_data_path = 's3://sagemaker-us-east-1-342942554664/aimodelpoll.csv'
s3_table_path = 's3://sagemaker-us-east-1-342942554664/table1/'

In [8]:
!aws s3 cp {s3_data_path} {s3_table_path}

copy: s3://sagemaker-us-east-1-342942554664/aimodelpoll.csv to s3://sagemaker-us-east-1-342942554664/table1/aimodelpoll.csv


In [9]:
#table1 - aimodelpoll.csv
table_name_csv = 'aimodelpoll'
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
ingest_create_athena_table_csv_passed = False
dataexplore = pd.read_csv('aimodelpoll.csv')
dataexplore.head()

,Question,Response,Overall,Age: 18-34,Age: 35-49,Age: 50-64,Age: 65+,Gender: Women,Gender: Men,College/No College: College,...,Race: Native American,Race: Asian,Race: Hispanic,Race: Black,Race: White,Occupation Category: Office Job,Occupation Category: Customer Facing Service Job,"Occupation Category: Manual, Not Customer Facing",Occupation Category: Education/Medicine,Occupation Category: Building Trades
0,AI Services Used,ChatGPT,0.348,0.488,0.407,0.328,0.171,0.341,0.355,0.414,...,0.330,0.402,0.375,0.352,0.337,0.399,0.342,0.299,0.360,0.317
1,AI Services Used,Claude,0.017,0.031,0.018,0.012,0.006,0.016,0.018,0.017,...,0.019,0.022,0.024,0.024,0.014,0.022,0.023,0.019,0.022,0.020
2,AI Services Used,Gemini,0.243,0.279,0.299,0.238,0.160,0.225,0.263,0.254,...,0.255,0.280,0.276,0.304,0.221,0.286,0.279,0.281,0.276,0.285
3,AI Services Used,Grok,0.038,0.063,0.043,0.029,0.018,0.028,0.049,0.035,...,0.038,0.043,0.047,0.039,0.036,0.043,0.040,0.044,0.038,0.048
4,AI Services Used,Copilot,0.122,0.115,0.140,0.145,0.092,0.108,0.137,0.166,...,0.112,0.152,0.115,0.129,0.119,0.149,0.102,0.109,0.134,0.121


In [10]:
dataexplore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 29 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Question                                          39 non-null     object 
 1   Response                                          37 non-null     object 
 2   Overall                                           39 non-null     float64
 3   Age: 18-34                                        39 non-null     float64
 4   Age: 35-49                                        39 non-null     float64
 5   Age: 50-64                                        39 non-null     float64
 6   Age: 65+                                          39 non-null     float64
 7   Gender: Women                                     39 non-null     float64
 8   Gender: Men                                       39 non-null     float64
 9   College/No College: Col

In [11]:
!aws s3 cp s3://sagemaker-us-east-1-342942554664/aimodelpoll.csv \
           s3://sagemaker-us-east-1-342942554664/table1/

copy: s3://sagemaker-us-east-1-342942554664/aimodelpoll.csv to s3://sagemaker-us-east-1-342942554664/table1/aimodelpoll.csv


In [12]:
!aws s3 ls s3://sagemaker-us-east-1-342942554664/table1/

2026-02-12 08:50:29       7979 aimodelpoll.csv


In [13]:
#sql table creation
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         Question STRING,
         Response STRING,
         Overall FLOAT,
         Age_18_34 FLOAT,
         Age_35_49 FLOAT,
         Age_50_64 FLOAT,
         Age_65_up FLOAT,
         Gender_Women FLOAT,
         Gender_Men FLOAT,
         College_No_College_College FLOAT,
         College_No_College_No_college FLOAT,
         Income_three_way_Lower_Income FLOAT,
         Income_three_way_Middle_Income FLOAT,
         Income_three_way_Upper_Income FLOAT,
         Generation_Gen_Z FLOAT,
         Generation_Millenial FLOAT,
         Generation_Gen_X FLOAT,
         Generation_Boomer FLOAT,
         Generation_Silent FLOAT,
         Race_Native_American FLOAT,
         Race_Asian FLOAT,
         Race_Hispanic FLOAT,
         Race_Black FLOAT,
         Race_White FLOAT,
         Occupation_Category_Office_Job FLOAT,
         Occupation_Category_Customer_Facing_Service_Job FLOAT,
         Occupation_Category_Manual_Not_Customer_Facing FLOAT,
         Occupation_Category_Education_Medicine FLOAT,
         Occupation_Category_Building_Trades FLOAT
) ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
  LINES TERMINATED BY '\\n' 
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_table_path
)

In [14]:
pd.read_sql(statement, conn)

/tmp/ipykernel_2654/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [15]:
statement = "SHOW TABLES in {}".format(database_name)
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_2654/84333453.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,aimodelpoll


In [16]:
statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, table_name_csv
)
print(statement)
df = pd.read_sql(statement, conn)
df.head()

SELECT * FROM dsoaws.aimodelpoll LIMIT 10


/tmp/ipykernel_2654/888292236.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,question,response,overall,age_18_34,age_35_49,age_50_64,age_65_up,gender_women,gender_men,college_no_college_college,...,race_native_american,race_asian,race_hispanic,race_black,race_white,occupation_category_office_job,occupation_category_customer_facing_service_job,occupation_category_manual_not_customer_facing,occupation_category_education_medicine,occupation_category_building_trades
0,AI Services Used,ChatGPT,0.348,0.488,0.407,0.328,0.171,0.341,0.355,0.414,...,0.330,0.402,0.375,0.352,0.337,0.399,0.342,0.299,0.360,0.317
1,AI Services Used,Claude,0.017,0.031,0.018,0.012,0.006,0.016,0.018,0.017,...,0.019,0.022,0.024,0.024,0.014,0.022,0.023,0.019,0.022,0.020
2,AI Services Used,Gemini,0.243,0.279,0.299,0.238,0.160,0.225,0.263,0.254,...,0.255,0.280,0.276,0.304,0.221,0.286,0.279,0.281,0.276,0.285
3,AI Services Used,Grok,0.038,0.063,0.043,0.029,0.018,0.028,0.049,0.035,...,0.038,0.043,0.047,0.039,0.036,0.043,0.040,0.044,0.038,0.048
4,AI Services Used,Copilot,0.122,0.115,0.140,0.145,0.092,0.108,0.137,0.166,...,0.112,0.152,0.115,0.129,0.119,0.149,0.102,0.109,0.134,0.121
